<a href="https://colab.research.google.com/github/oskarklima/M7DataSP/blob/main/assignment08_qwen3__14b_reasoning_conversational.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth your local device, follow [our guide](https://docs.unsloth.ai/get-started/install-and-update). This notebook is licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News


Unsloth's [Docker image](https://hub.docker.com/r/unsloth/unsloth) is here! Start training with no setup & environment issues. [Read our Guide](https://docs.unsloth.ai/new/how-to-train-llms-with-unsloth-and-docker).

[gpt-oss RL](https://docs.unsloth.ai/new/gpt-oss-reinforcement-learning) is now supported with the fastest inference & lowest VRAM. Try our [new notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/gpt-oss-(20B)-GRPO.ipynb) which creates kernels!

Introducing [Vision](https://docs.unsloth.ai/new/vision-reinforcement-learning-vlm-rl) and [Standby](https://docs.unsloth.ai/basics/memory-efficient-rl) for RL! Train Qwen, Gemma etc. VLMs with GSPO - even faster with less VRAM.

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [2]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

### Unsloth

In [3]:
from unsloth import FastLanguageModel
import torch

fourbit_models = [
    "unsloth/Qwen3-1.7B-unsloth-bnb-4bit", # Qwen 14B 2x faster
    "unsloth/Qwen3-4B-unsloth-bnb-4bit",
    "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    "unsloth/Qwen3-14B-unsloth-bnb-4bit",
    "unsloth/Qwen3-32B-unsloth-bnb-4bit",

    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/Phi-4",
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/orpheus-3b-0.1-ft-unsloth-bnb-4bit" # [NEW] We support TTS models!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-14B",
    max_seq_length = 2048,   # Context length - can be longer, but uses more memory
    load_in_4bit = True,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False, # We have full finetuning now!
    # token = "hf_...",      # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.2: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.59G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)

Unsloth 2025.11.2 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


<a name="Data"></a>
### Data Prep
Qwen3 has both reasoning and a non reasoning mode. So, we should use 2 datasets:

1. We use the [Open Math Reasoning]() dataset which was used to win the [AIMO](https://www.kaggle.com/competitions/ai-mathematical-olympiad-progress-prize-2/leaderboard) (AI Mathematical Olympiad - Progress Prize 2) challenge! We sample 10% of verifiable reasoning traces that used DeepSeek R1, and whicht got > 95% accuracy.

2. We also leverage [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we need to convert it to HuggingFace's normal multiturn format as well.

In [5]:
from datasets import load_dataset
reasoning_dataset = load_dataset("unsloth/OpenMathReasoning-mini", split = "cot")
non_reasoning_dataset = load_dataset("mlabonne/FineTome-100k", split = "train")

README.md:   0%|          | 0.00/603 [00:00<?, ?B/s]

data/cot-00000-of-00001.parquet:   0%|          | 0.00/106M [00:00<?, ?B/s]

Generating cot split:   0%|          | 0/19252 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Let's see the structure of both datasets:

In [6]:
reasoning_dataset

Dataset({
    features: ['expected_answer', 'problem_type', 'problem_source', 'generation_model', 'pass_rate_72b_tir', 'problem', 'generated_solution', 'inference_mode'],
    num_rows: 19252
})

In [7]:
non_reasoning_dataset

Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 100000
})

We now convert the reasoning dataset into conversational format:

In [8]:
def generate_conversation(examples):
    problems  = examples["problem"]
    solutions = examples["generated_solution"]
    conversations = []
    for problem, solution in zip(problems, solutions):
        conversations.append([
            {"role" : "user",      "content" : problem},
            {"role" : "assistant", "content" : solution},
        ])
    return { "conversations": conversations, }

In [9]:
reasoning_conversations = tokenizer.apply_chat_template(
    reasoning_dataset.map(generate_conversation, batched = True)["conversations"],
    tokenize = False,
)

Map:   0%|          | 0/19252 [00:00<?, ? examples/s]

Let's see the first transformed row:

In [10]:
reasoning_conversations[0]

"<|im_start|>user\nGiven $\\sqrt{x^2+165}-\\sqrt{x^2-52}=7$ and $x$ is positive, find all possible values of $x$.<|im_end|>\n<|im_start|>assistant\n<think>\nOkay, let's see. I need to solve the equation √(x² + 165) - √(x² - 52) = 7, and find all positive values of x. Hmm, radicals can be tricky, but maybe if I can eliminate the square roots by squaring both sides. Let me try that.\n\nFirst, let me write down the equation again to make sure I have it right:\n\n√(x² + 165) - √(x² - 52) = 7.\n\nOkay, so the idea is to isolate one of the radicals and then square both sides. Let me try moving the second radical to the other side:\n\n√(x² + 165) = 7 + √(x² - 52).\n\nNow, if I square both sides, maybe I can get rid of the square roots. Let's do that:\n\n(√(x² + 165))² = (7 + √(x² - 52))².\n\nSimplifying the left side:\n\nx² + 165 = 49 + 14√(x² - 52) + (√(x² - 52))².\n\nThe right side is expanded using the formula (a + b)² = a² + 2ab + b². So the right side becomes 7² + 2*7*√(x² - 52) + (√(x² 

Next we take the non reasoning dataset and convert it to conversational format as well.

We have to use Unsloth's `standardize_sharegpt` function to fix up the format of the dataset first.

In [11]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(non_reasoning_dataset)

non_reasoning_conversations = tokenizer.apply_chat_template(
    dataset["conversations"],
    tokenize = False,
)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

Let's see the first row

In [12]:
non_reasoning_conversations[0]

'<|im_start|>user\nExplain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.<|im_end|>\n<|im_start|>assistant\n<think>\n\n</th

Now let's see how long both datasets are:

In [13]:
print(len(reasoning_conversations))
print(len(non_reasoning_conversations))

19252
100000


The non reasoning dataset is much longer. Let's assume we want the model to retain some reasoning capabilities, but we specifically want a chat model.

Let's define a ratio of chat only data. The goal is to define some mixture of both sets of data.

Let's select 75% reasoning and 25% chat based:

In [14]:
chat_percentage = 0.75

Let's sample the reasoning dataset by 75% (or whatever is 100% - chat_percentage)

In [15]:
import pandas as pd
non_reasoning_subset = pd.Series(non_reasoning_conversations)
non_reasoning_subset = non_reasoning_subset.sample(
    int(len(reasoning_conversations)*(chat_percentage/(1 - chat_percentage))),
    random_state = 2407,
)
print(len(reasoning_conversations))
print(len(non_reasoning_subset))
print(len(non_reasoning_subset) / (len(non_reasoning_subset) + len(reasoning_conversations)))

19252
57756
0.75


Add our dataset

In [17]:
from google.colab import files
import pandas as pd


uploaded = files.upload()

if 'text_clean.txt' not in uploaded:
    raise ValueError("Soubor 'text_clean.txt' nebyl nahrán.")
else:
    print("Soubor 'text_clean.txt' úspěšně nahrán.")

with open('text_clean.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

print(lines)
my_single_conversation = []
current_content = ""
current_role = None

for line in lines[1:]:
    line = line.strip()
    if not line:
        continue

    if line.startswith("Ž:"):
        if current_role == "assistant" and current_content:
            my_single_conversation.append({"role": "assistant", "content": current_content.strip()})

        current_role = "user"
        current_content = line[2:].strip()

    elif line.startswith("U:"):
        if current_role == "user" and current_content:
            my_single_conversation.append({"role": "user", "content": current_content.strip()})

        current_role = "assistant"
        current_content = line[2:].strip()

    else:
        if current_role is not None:
            current_content += " " + line

if current_role == "user" and current_content:
    my_single_conversation.append({"role": "user", "content": current_content.strip()})
elif current_role == "assistant" and current_content:
    my_single_conversation.append({"role": "assistant", "content": current_content.strip()})

my_formatted_dataset = [my_single_conversation]

my_conversations_text = tokenizer.apply_chat_template(
    my_formatted_dataset,
    tokenize = False,
)

my_conversations_series = pd.Series(my_conversations_text)
print(f"Úspěšně zpracován 1 dlouhý dialog z 'text_clean.txt'")

Saving text_clean.txt to text_clean.txt
Soubor 'text_clean.txt' úspěšně nahrán.
['RNDr. Viera Vodičková\n', 'U: O kružnici si už určite počul.\n', 'Ž: Samozrejme. S kružnicou sa stretávame všade. Je to také „koliesko. A teraz vážne. Kruž-\n', 'nica je daná stredom a polomerom. A všetky body na kružnici sú od stredu vzdialené o tento\n', 'U: Ja to už len upresním formou definície. Kružnicou k so stredom S a polomerom r\n', 'nazývame množinu všetkých bodov v rovine, ktorých vzdialenosť od stredu S\n', 'je rovná polomeru r. Symbolický zápis si pozri v rámčeku. Rovina, v ktorej pracujeme,\n', 'sa nazýva dvojrozmerný Euklidovský priestor a označuje sa E .\n', 'k(S;r) = {X ∈ E ;|XS| = r;r ∈ R+} Podobne sa definuje aj kruh.\n', 'Ž: Myslíte to „vnútro? Skúsim to sám. Kruhom K so stredom S a polomerom r na-\n', 'zývame množinu všetkých bodov v rovine, ktorých vzdialenosť od stredu S je\n', 'menšia alebo rovná polomeru r.\n', 'U: Budeme sa zaoberať tým ako kružnicu, prípadne kruh, vyjadríme anal

Finally combine both datasets:

In [18]:
data = pd.concat([
    pd.Series(reasoning_conversations),
    pd.Series(non_reasoning_subset),
    my_conversations_series
])
data.name = "text"

from datasets import Dataset
combined_dataset = Dataset.from_pandas(pd.DataFrame(data))
combined_dataset = combined_dataset.shuffle(seed = 3407)

print(f"Finální dataset má celkem {len(combined_dataset)} příkladů.")

Finální dataset má celkem 77009 příkladů.


<a name="Train"></a>
### Train the model
Now let's train our model. We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [19]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = combined_dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        #num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 5,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use TrackIO/WandB etc
        dataset_num_proc = 1, # Potřeba přidat
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/77009 [00:00<?, ? examples/s]

In [20]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
11.898 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [21]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 77,009 | Num Epochs = 1 | Total steps = 5
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 128,450,560 of 14,896,757,760 (0.86% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.114800
2,0.940500
3,0.677300
4,1.403400
5,0.718200


In [22]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

226.4252 seconds used for training.
3.77 minutes used for training.
Peak reserved memory = 14.578 GB.
Peak reserved memory for training = 2.68 GB.
Peak reserved memory % of max memory = 98.894 %.
Peak reserved memory for training % of max memory = 18.181 %.


<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Qwen-3` team, the recommended settings for reasoning inference are `temperature = 0.6, top_p = 0.95, top_k = 20`

For normal chat based inference, `temperature = 0.7, top_p = 0.8, top_k = 20`

In [23]:
messages = [
    {"role" : "user", "content" : "Solve (x + 2)^2 = 0."}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = False, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 256, # Increase for longer outputs!
    temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

We are given the equation:

$$
(x + 2)^2 = 0
$$

### Step 1: Take the square root of both sides

$$
\sqrt{(x + 2)^2} = \sqrt{0}
$$

$$
|x + 2| = 0
$$

### Step 2: Solve the absolute value equation

$$
x + 2 = 0
$$

$$
x = -2
$$

### ✅ Final Answer:

$$
\boxed{-2}
$$

This is the only solution to the equation.<|im_end|>


In [24]:
messages = [
    {"role" : "user", "content" : "Solve (x + 2)^2 = 0."}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = True, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 1024, # Increase for longer outputs!
    temperature = 0.6, top_p = 0.95, top_k = 20, # For thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<think>
Okay, so I need to solve the equation (x + 2)^2 = 0. Hmm, let me think. I remember that when you have something squared equals zero, there's probably only one solution because squaring any real number gives a non-negative result, and the only way that squared term can be zero is if the inside is zero. Let me verify that.

First, let me rewrite the equation to make it clearer. The equation is (x + 2) squared equals zero. So, mathematically, that's (x + 2)^2 = 0. To solve for x, I need to get x by itself. Since it's squared, maybe I can take the square root of both sides? Wait, but taking the square root of both sides usually gives me two solutions, right? Like, if I have x^2 = a, then x = ±√a. But here, the right side is zero. So, if I take the square root of both sides, I would have sqrt((x + 2)^2) = sqrt(0). The square root of (x + 2)^2 is |x + 2|, and the square root of zero is zero. So, |x + 2| = 0. 

But the absolute value of something is zero only when that something is ze

In [25]:
from transformers import TextStreamer

messages = [
    {"role" : "user", "content" : "Co je kružnice?"}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True,
    enable_thinking = False,
)

_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 512,
    temperature = 0.7, top_p = 0.8, top_k = 20,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Kružnice je **geometrická množina všech bodů v rovině, které mají stejnou vzdálenost od daného bodu**, který se nazývá **střed kružnice**. Tato vzdálenost se nazývá **poloměr kružnice**.

### Základní pojmy:
- **Střed kružnice (S)** – bod, od kterého mají všechny body kružnice stejnou vzdálenost.
- **Poloměr (r)** – vzdálenost mezi středem a libovolným bodem na kružnici.
- **Diametr (d)** – úsečka, která prochází středem kružnice a jejími dvěma body. Délka diametru je dvojnásobek délky poloměru: $ d = 2r $.

### Vzorec pro kružnici:
Kružnici můžeme popsat rovnicí v rovině. Pokud je střed kružnice v bodě $ S = (x_0, y_0) $ a její poloměr je $ r $, pak rovnice kružnice je:

$$
(x - x_0)^2 + (y - y_0)^2 = r^2
$$

### Vlastnosti kružnice:
- Každá kružnice má **neomezeně mnoho bodů**.
- Kružnice je **uzavřená křivka**.
- Kružnice má **středovou souměrnost** i **osovou souměrnost** (každá přímka procházející středem je osou souměrnosti).

### Příklad:
Kružnice se středem v bodě $ (2, 3) $ a 

In [26]:
messages = [
    {"role" : "user", "content" : "Kolik existuje přirozených čísel n menších než 4444 takových, že existují přirozená čísla a, b tak, že 4444a+2222b dává zbytek 1 po dělení n?"}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True,
    enable_thinking = False,
)

_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 512,
    temperature = 0.7, top_p = 0.8, top_k = 20,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Chceme zjistit, kolik existuje **přirozených čísel $ n < 4444 $**, pro která existují **přirozená čísla $ a, b $** tak, že:

$$
4444a + 2222b \equiv 1 \pmod{n}
$$

---

### Krok 1: Upravme výraz

Zkusme vyjádřit výraz $ 4444a + 2222b $ jako:

$$
4444a + 2222b = 2222(2a + b)
$$

Takže:

$$
2222(2a + b) \equiv 1 \pmod{n}
$$

To znamená, že **$ 2222(2a + b) \equiv 1 \pmod{n} $**.

---

### Krok 2: Znamená to, že 2222 má inverzi modulo $ n $

Pro dané $ n $, pokud existuje $ x \in \mathbb{Z} $ tak, že:

$$
2222x \equiv 1 \pmod{n}
$$

pak $ x $ je **inverze čísla 2222 modulo $ n $**. A to znamená, že:

$$
\gcd(2222, n) = 1
$$

Protože inverze existuje **pouze pokud $ \gcd(2222, n) = 1 $**.

---

### Krok 3: Zjistíme, pro která $ n < 4444 $ platí $ \gcd(2222, n) = 1 $

Chceme tedy zjistit, kolik čísel $ n < 4444 $ je **nepříbuzných s 2222**, tedy:

$$
\text{počet } n < 4444, \text{ pro která } \gcd(2222, n) = 1
$$

To je přesně **Eulerova funkce $ \phi(2222) $**


In [29]:
messages = [
    {"role" : "user", "content" : "Dnes se chci dozvědět nové věci o matematice."},
    {"role" : "assistant", "content" : "Co by tě zajímalo studente?"},
    {"role" : "user", "content" : "Třeba nějaká lineární algebra."}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True,
    enable_thinking = False,
)

_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 512,
    temperature = 0.7, top_p = 0.8, top_k = 20,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Skvělé! Lineární algebra je skvělá oblast matematiky, která má obrovské využití jak v teoretické matematice, tak v praktických oborech jako je fyzika, inženýrství, počítačová grafika, strojové učení a mnohé další.

Zkusíme si dnes probrat **základy lineární algebry** – začneme od základních pojmů a postupně se budeme zvyšovat v obtížnosti. Těším se na tvoje otázky a zpětnou vazbu.

---

### 🔹 1. Co je to **vektor**?

**Vektor** je matematický objekt, který má **velikost a směr**. Můžeš si ho představit jako šipku v rovině nebo v prostoru.

Příklad:
- V rovině: $\vec{v} = \begin{pmatrix} 2 \\ 3 \end{pmatrix}$
- V prostoru: $\vec{v} = \begin{pmatrix} 1 \\ -2 \\ 4 \end{pmatrix}$

---

### 🔹 2. Co je to **matice**?

**Matice** je obdélníková tabulka čísel, která se používá k reprezentaci systémů lineárních rovnic, transformací, a mnoha jiných věcí.

Příklad:
$$
A = \begin{pmatrix}
1 & 2 & 3 \\
4 & 5 & 6 \\
7 & 8 & 9
\end{pmatrix}
$$

- Tato matice má 3 řádky a 3 sloupce → je to **čtvercová

In [30]:
messages = [
    {"role" : "user", "content" : "Dnes se chci dozvědět nové věci o matematice."},
    {"role" : "assistant", "content" : "Co by tě zajímalo studente?"},
    {"role" : "user", "content" : "Třeba nějaká lineární algebra."},
    {"role" : "assistant", "content" : "Víš třeba co je to vektor?"},
    {"role" : "assistant", "content" : "Něco tuším, ale nejsem si jist. Můžeme to probrat? Není to seznam čísel?"},
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True,
    enable_thinking = False,
)

_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 256,
    temperature = 0.7, top_p = 0.8, top_k = 20,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Ano, **vektor** je skvělé téma! 😊

V matematice je **vektor** obvykle definován jako **usměrněná úsečka**, tedy úsečka, která má nejen délku, ale i směr. Vektor můžeš představit jako šipku, který ukazuje, kam se máš pohybovat – například o 3 jednotky doprava a 4 jednotky nahoru.

### Co je to vektor v praxi?
- V 2D (dvourozměrném prostoru) může vektor vypadat jako dvojice čísel:  
  $$
  \vec{v} = \begin{pmatrix} 3 \\ 4 \end{pmatrix}
  $$
  Tento vektor znamená: „Pohybuj se 3 jednotky do prava a 4 jednotky nahoru.“

- V 3D (třírozměrném prostoru) může vek


In [31]:
from transformers import TextStreamer

messages = [
    {
        "role": "system",
        "content": "Jsi Učitel matematiky. Oslovuj uživatele 'Žáku' nebo 'studente'. Tvoje odpovědi jsou ve stylu dialogu mezi učitelem a žákem."
    },

    {"role" : "user", "content" : "Dnes se chci dozvědět nové věci o matematice."},
    {"role" : "assistant", "content" : "Co by tě zajímalo studente?"},
    {"role" : "user", "content" : "Třeba nějaká lineární algebra."}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True,
    enable_thinking = False,
)

_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 512,
    temperature = 0.7, top_p = 0.8, top_k = 20,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Skvělé, studente! Lineární algebra je opravdu důležitá oblast matematiky, která se zabývá vektory, maticemi a lineárními transformacemi. Začneme základy – co bys chtěl vědět o vektorech nebo maticích?

Můžeme si například vysvětlit, co to vůbec je vektor, nebo se podívat na to, jak se sčítají a násobí. Co ti spíš vyhovuje?<|im_end|>


In [33]:
from transformers import TextStreamer

messages = [
    {
        "role": "system",
        "content": "Jsi Učitel matematiky. Oslovuj uživatele 'Žáku' nebo 'studente'. Tvoje odpovědi jsou ve stylu dialogu mezi učitelem a žákem. Nekonči moc brzo ať je dialog delší."
    },

    {"role" : "user", "content" : "Dnes se chci dozvědět nové věci o matematice."},
    {"role" : "assistant", "content" : "Co by tě zajímalo studente?"},
    {"role" : "user", "content" : "Třeba nějaká lineární algebra."}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True,
    enable_thinking = False,
)

_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 512,
    temperature = 0.7, top_p = 0.8, top_k = 20,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Skvělé, lineární algebra je skvělá oblast, která má mnoho praktických aplikací. Začneme základy – co si myslíš, co je to vlastně **vektor**?

Můžeš mi říct, jak bys si to představoval?<|im_end|>


In [35]:
from transformers import TextStreamer

messages = [
    {
        "role": "system",
        "content": "Jsi Učitel matematiky. Oslovuj uživatele 'Žáku' nebo 'studente'. Tvoje odpovědi jsou ve stylu dialogu mezi učitelem a žákem. Nekonči moc brzo ať je dialog delší. Nesmíš použít <|im_end|>"
    },

    {"role" : "user", "content" : "Dnes se chci dozvědět nové věci o matematice."},
    {"role" : "assistant", "content" : "Co by tě zajímalo studente?"},
    {"role" : "user", "content" : "Třeba nějaká lineární algebra."}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True,
    enable_thinking = False,
)

_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 512,
    temperature = 0.6, top_p = 0.95, top_k = 20,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Dobrá, studente, lineární algebra je skvělý téma. Začneme základy. Co si myslíš, co je to vlastně matice?

Můžeš mi říct, co si pod tím slovem představuješ?<|im_end|>
